In [8]:
import numpy as np
import pandas as pd

# Load the data from the .npy file
data = np.load('development_numpy/development.npy', allow_pickle=True)

# Load metadata from development.csv
metadata = pd.read_csv('metadata/development.csv')

# Load feature names mapping from idx_to_feature_names.csv
feature_names_mapping = pd.read_csv('metadata/idx_to_feature_name.csv')

# Extract labels and speaker IDs from metadata
labels = metadata['word'].values  # Adjusted to use 'word' column as class labels
speaker_ids = metadata['speaker_id'].values

# Assuming the snippet IDs are sequential integers starting from 0
snippet_ids = np.arange(len(metadata))

# Check the shapes of the data and labels
print('Data shape:', data.shape)
print('Labels shape:', labels.shape)


Data shape: (45296, 175, 44)
Labels shape: (45296,)


In [9]:
from sklearn.model_selection import train_test_split

# Subsample the data (e.g., 10% of the original dataset)
subsample_fraction = 0.1
X_subsample, _, y_subsample, _, snippet_ids_subsample, _ = train_test_split(
    data, labels, snippet_ids, train_size=subsample_fraction, random_state=42)

# Check the shapes of the subsampled data
print('Subsampled data shape:', X_subsample.shape)
print('Subsampled labels shape:', y_subsample.shape)

# Flatten the feature dimensions (n_features * n_time) for use with SVM
n_samples, n_features, n_time = X_subsample.shape
X_subsample_flat = X_subsample.reshape(n_samples, n_features * n_time)

# Verify reshaped data
print('Reshaped subsampled data shape:', X_subsample_flat.shape)


Subsampled data shape: (4529, 175, 44)
Subsampled labels shape: (4529,)
Reshaped subsampled data shape: (4529, 7700)


In [10]:
# Split the subsampled data into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test, snippet_ids_train_val, snippet_ids_test = train_test_split(
    X_subsample_flat, y_subsample, snippet_ids_subsample, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val, snippet_ids_train, snippet_ids_val = train_test_split(
    X_train_val, y_train_val, snippet_ids_train_val, test_size=0.5, random_state=42)

# Check the shapes after splitting
print('Training data shape:', X_train.shape)
print('Training labels shape:', y_train.shape)
print('Validation data shape:', X_val.shape)
print('Validation labels shape:', y_val.shape)
print('Test data shape:', X_test.shape)
print('Test labels shape:', y_test.shape)


Training data shape: (1585, 7700)
Training labels shape: (1585,)
Validation data shape: (1585, 7700)
Validation labels shape: (1585,)
Test data shape: (1359, 7700)
Test labels shape: (1359,)


In [11]:
from sklearn.preprocessing import StandardScaler

# Standardize the features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Verify the scaling
print('Training data mean (scaled):', np.mean(X_train_scaled))
print('Training data std (scaled):', np.std(X_train_scaled))


Training data mean (scaled): -4.266342060428542e-18
Training data std (scaled): 1.0


In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define the SVM model
svm = SVC()

# Set up the hyperparameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Set up GridSearchCV with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

# Output the best hyperparameters and best score
print('Best hyperparameters (SVM):', grid_search.best_params_)
print('Best cross-validation score (SVM):', grid_search.best_score_)

# Evaluate the best SVM model on the validation and test sets
best_svm = grid_search.best_estimator_
val_score_svm = best_svm.score(X_val_scaled, y_val)
test_score_svm = best_svm.score(X_test_scaled, y_test)
print('Validation accuracy (SVM):', val_score_svm)
print('Test accuracy (SVM):', test_score_svm)


Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=  14.5s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=  14.5s
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time=  14.5s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=  14.6s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=  17.1s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=  17.2s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=  17.3s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=  19.8s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=  19.8s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=  19.8s
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time=  14.7s
[CV] END ...................C=0.1, gamma=auto, k